# wps_CI_on_demand

#### wps_CI is a process that runs the [ci.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CI.R#L235) (Climate Imprint) function of the [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package. This notebook uses either the PNWNAmet daily gridded observations or the CMIP6 downscaled data as the `gcm_file` parameter and the PRISM monthly climatologies as the `obs_file` parameter. By supplying climatologies, this skips the computation of the climatologies during Climate Imprint. To get started, first instantiate the client. Here, the client will try to connect to a `chickadee` instance using the `url` parameter. This instantiation also takes advantage of asynchronous execution

## 1. Setup chickadee instance and input parameters

I. Import Python packages used throughout the notebook.

In [1]:
import os
import shapely.geometry
import numpy as np
from birdy import WPSClient
from tempfile import NamedTemporaryFile
from netCDF4 import Dataset
from datetime import date
from requests_html import HTMLSession
from ipywidgets import *
from ipyleaflet import *

II. Connect to the chickadee instance given by the `url`.

In [2]:
# NBVAL_IGNORE_OUTPUT
url = "http://docker-dev03.pcic.uvic.ca:30102"
print(f"Using chickadee on {url}")
chickadee = WPSClient(url, progress=True)

Using chickadee on http://docker-dev03.pcic.uvic.ca:30102


III. Display help for individual processes by using the ? command (ex. bird.process?). We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# NBVAL_IGNORE_OUTPUT
chickadee.ci?

Signature:
chickadee.ci(
    gcm_file,
    obs_file,
    gcm_varname=None,
    obs_varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
)
Docstring:
Climate Imprint (CI) downscaling

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
units_boo

IV. Set up an interactive map to initialize the inputs. These inputs are the <b>center point</b> of the 2.5 degree by 2.5 degree subdomain of the target region and 3 degree by 3 degree subdomain of the GCM region, the <b>climate variable</b> from the GCM input to downscale, the <b>dataset</b> to use as the GCM input (where choosing the CMIP6 data allows one to further select the <b>downscaling technique</b>, the <b>model</b>, the <b>CanESM5 run</b> if applicable, and the <b>emissions scenario</b>), and the <b>climatological period</b> of the target data. Some inputs are obtained from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html).

In [4]:
sub_layers = LayerGroup()
thredds_catalog = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog/datasets"

def get_subdomain(lat_min, lat_max, lon_min, lon_max, color, name):
    coords = [(lat_min, lon_min), (lat_max, lon_max)]
    return Rectangle(bounds=coords, color=color, name=name, draggable=True)

def get_models():
    session = HTMLSession()
    r = session.get(f"{thredds_catalog}/storage/data/climate/downscale/BCCAQ2/CMIP6_BCCAQv2/catalog.html")
    models = []
    exclude = ["CMIP6_BCCAQv2", "CWEC2020_Factors/", "Degree_Climatologies/", "Ensemble_Averages/", "nobackup/", "--", ""]
    for tt in r.html.find("tt"):
        if tt.text not in exclude:
            models.append(tt.text[:-1])
    models.sort()
    return models

def handle_dataset_change(change):
    technique.disabled = not technique.disabled
    model.disabled = not model.disabled
    scenario.disabled = not scenario.disabled

def handle_model_change(change):
    if model.value == "CanESM5":
        canesm5_run.disabled = False
    else:
        canesm5_run.disabled = True
        
def handle_interact(**kwargs):
    point = (round(kwargs.get("coordinates")[0], 5), round(kwargs.get("coordinates")[1], 5))
    center_hover.value = str(point)
    if kwargs.get("type") == "click":
        print(sub_layers)
        print("Hello")
        # Remove previous center point and subdomains
        for layer in sub_layers.layers:
            sub_layers.remove_layer(layer)

        # Add new subdomains
        m.center_point = point
        center.value = str(m.center_point)
        center_marker = Marker(location=m.center_point, name="Marker")
        
        m.lat_min_obs, m.lat_max_obs = (m.center_point[0] - 1.25, m.center_point[0] + 1.25)
        m.lon_min_obs, m.lon_max_obs = (m.center_point[1] - 1.25, m.center_point[1] + 1.25)
        m.lat_min_gcm, m.lat_max_gcm = (m.center_point[0] - 1.5, m.center_point[0] + 1.5)
        m.lon_min_gcm, m.lon_max_gcm = (m.center_point[1] - 1.5, m.center_point[1] + 1.5)

        gcm_subdomain = get_subdomain(m.lat_min_gcm, m.lat_max_gcm, m.lon_min_gcm, m.lon_max_gcm, "blue", "GCM")
        obs_subdomain = get_subdomain(m.lat_min_obs, m.lat_max_obs, m.lon_min_obs, m.lon_max_obs, "red", "Obs")

        sub_layers.add_layer(center_marker)
        sub_layers.add_layer(gcm_subdomain)
        sub_layers.add_layer(obs_subdomain)
        m.add_layer(sub_layers)

In [5]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = "Default"

m = Map(
    basemap=mapnik,
    center=(53.5, -120),
    zoom=5,
    layout=Layout(height="600px"),
)
m.on_interaction(handle_interact)
m.center_point = ()
    
legend = LegendControl({"GCM": "blue", "Obs": "red"}, name="Subdomains", position="topright")
m.add_control(legend)

center_hover = Text(value="", placeholder="") 
center = Text(value="", placeholder="", description="Center:")
vars = RadioButtons(options=["pr", "tasmax", "tasmin"], description="Climate variable:")
dataset = RadioButtons(options=["PNWNAmet", "CMIP6"], description="Dataset:")
dataset.observe(handle_dataset_change)

technique = RadioButtons(options=["BCCAQv2", "MBCn"], description="CMIP6 downscaling technique:", disabled = True)
model = Dropdown(options=get_models(), description="CMIP6 model:", disabled = True)
model.style.description_width = "100px"
model.observe(handle_model_change)

canesm5_runs = ["r" + str(r) + "i1p2f1" for r in range(1, 11)]
canesm5_run = Dropdown(options=canesm5_runs, description="CanESM5 run:", disabled = True)
canesm5_run.style.description_width = "100px"

scenario = RadioButtons(options=[("SSP1-2.6", "ssp126"), ("SSP2-4.5", "ssp245"), ("SSP5-8.5", "ssp585")], description="CMIP6 emissions scenario:", disabled = True)
period = RadioButtons(options=["197101-200012", "198101-201012"], description="Climatological period:")

box_layout = Layout(display='flex',
                flex_flow = 'column', 
                width='110%',
                align_items = 'center')
control_box = Box(children = [center_hover, center, vars, dataset, technique, model, canesm5_run, scenario, period], layout=box_layout)
AppLayout(center = m, right_sidebar = control_box, align_items = 'center')

AppLayout(children=(Box(children=(Text(value='', placeholder=''), Text(value='', description='Center:', placeh…

V. Obtain the input data files from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html) and examine their structures.

In [7]:
thredds_base = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets"
data_vars = {"pr": "pr", "tasmax": "tmax", "tasmin": "tmin"}

gcm_var = vars.value
obs_var = data_vars[gcm_var]

if dataset.value == "PNWNAmet":
    gcm_file = f"{thredds_base}/storage/data/projects/dataportal/data/vic-gen2-forcing/PNWNAmet_{gcm_var}_invert_lat.nc"
else:
    if technique.value == "BCCAQv2":
        technique_dir = "BCCAQ2"
        model_dir = model.value
    else:
        technique_dir = "MBCn"
        model_dir = model.value + "_10"
    model_catalog = f"{thredds_catalog}/storage/data/climate/downscale/{technique_dir}/CMIP6_{technique.value}/{model_dir}/catalog.html"
    
    session = HTMLSession()
    r = session.get(model_catalog)
    for tt in r.html.find("tt"):
        file = tt.text
        if (gcm_var in file) and (scenario.value in file):
            if (model.value == "CanESM5") and (canesm5_run.value not in file):
                continue
            break
    gcm_file = f"{thredds_base}/storage/data/climate/downscale/{technique_dir}/CMIP6_{technique.value}/{model_dir}/{file}"
        
obs_file = f"{thredds_base}/storage/data/climate/PRISM/dataportal/{obs_var}_monClim_PRISM_historical_run1_{period.value}.nc"
print("GCM File: " + gcm_file + "\n")
print("Obs File: " + obs_file + "\n")
gcm_dataset = Dataset(gcm_file)
obs_dataset = Dataset(obs_file)
print("GCM Structure: " + str(gcm_dataset.dimensions.items()) + "\n")
print("Obs Structure: " + str(obs_dataset.dimensions.items()) + "\n")

GCM File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/downscale/MBCn/CMIP6_MBCn/BCC-CSM2-MR_10/tasmax_day_MBCn+PCIC-Blend_BCC-CSM2-MR_historical+ssp245_r1i1p1f1_gn_19500101-21001231.nc

Obs File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/PRISM/dataportal/tmax_monClim_PRISM_historical_run1_197101-200012.nc

GCM Structure: dict_items([('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 510), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1068), ('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 55115)])

Obs Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 13), ('bnds', <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 1680), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 3241

VI. Store the datasets' latitudes and longitudes into variables and examine their values.

In [35]:
gcm_lats = gcm_dataset.variables["lat"][:]
gcm_lons = gcm_dataset.variables["lon"][:]
obs_lats = obs_dataset.variables["lat"][:]
obs_lons = obs_dataset.variables["lon"][:]

In [36]:
gcm_lats

masked_array(data=[41.04166794, 41.125     , 41.20833206, 41.29166794,
                   41.375     , 41.45833206, 41.54166794, 41.625     ,
                   41.70833206, 41.79166794, 41.875     , 41.95833206,
                   42.04166794, 42.125     , 42.20833206, 42.29166794,
                   42.375     , 42.45833206, 42.54166794, 42.625     ,
                   42.70833206, 42.79166794, 42.875     , 42.95833206,
                   43.04166794, 43.125     , 43.20833206, 43.29166794,
                   43.375     , 43.45833206, 43.54166794, 43.625     ,
                   43.70833206, 43.79166794, 43.875     , 43.95833206,
                   44.04166794, 44.125     , 44.20833206, 44.29166794,
                   44.375     , 44.45833206, 44.54166794, 44.625     ,
                   44.70833206, 44.79166794, 44.875     , 44.95833206,
                   45.04166794, 45.125     , 45.20833206, 45.29166794,
                   45.375     , 45.45833206, 45.54166794, 45.625     ,
      

In [37]:
gcm_lons

masked_array(data=[-140.95832825, -140.875     , -140.79167175, ...,
                    -52.20832825,  -52.12499619,  -52.04166031],
             mask=False,
       fill_value=1e+20)

In [38]:
obs_lats

masked_array(data=[48.        , 48.00833333, 48.01666667, ...,
                   61.975     , 61.98333333, 61.99166667],
             mask=False,
       fill_value=1e+20)

In [39]:
obs_lons

masked_array(data=[-140.        , -139.99166667, -139.98333333, ...,
                   -113.01666667, -113.00833333, -113.        ],
             mask=False,
       fill_value=1e+20)

VII. Request a subset of each dataset based on each subdomain. This is done by determining the indices corresponding to the min/max lat and lon values and using those in the THREDDS requests. The full time range for each dataset is also used.

In [40]:
def get_index_range(arr, min_val, max_val):
    """Compute the indices in an array that correspond to the array's values
    closest to desired min/max values."""        
    min_index = np.argmin(np.abs(arr - min_val))
    max_index = np.argmin(np.abs(arr - max_val))
    return (min_index, max_index)

In [41]:
gcm_lat_indices = get_index_range(gcm_lats, m.lat_min_gcm, m.lat_max_gcm)
gcm_lon_indices = get_index_range(gcm_lons, m.lon_min_gcm, m.lon_max_gcm)
obs_lat_indices = get_index_range(obs_lats, m.lat_min_obs, m.lat_max_obs)
obs_lon_indices = get_index_range(obs_lons, m.lon_min_obs, m.lon_max_obs)

In [42]:
print("GCM lat index range: " + str(gcm_lat_indices))
print("GCM lon index range: " + str(gcm_lon_indices))
print("Obs lat index range: " + str(obs_lat_indices))
print("Obs lon index range: " + str(obs_lon_indices))

GCM lat index range: (181, 217)
GCM lon index range: (133, 169)
Obs lat index range: (1002, 1302)
Obs lon index range: (1246, 1546)


In [43]:
gcm_lat_range = f"[{gcm_lat_indices[0]}:{gcm_lat_indices[1]}]"
gcm_lon_range = f"[{gcm_lon_indices[0]}:{gcm_lon_indices[1]}]"
obs_lat_range = f"[{obs_lat_indices[0]}:{obs_lat_indices[1]}]"
obs_lon_range = f"[{obs_lon_indices[0]}:{obs_lon_indices[1]}]"

In [44]:
gcm_ntime = len(gcm_dataset.variables["time"][:])
gcm_time_range = f"[0:{gcm_ntime - 1}]"
obs_ntime = len(obs_dataset.variables["time"][:])
obs_time_range = f"[0:{obs_ntime - 1}]"

In [45]:
gcm_subset_file = f"{gcm_file}?time,lat{gcm_lat_range},lon{gcm_lon_range},{gcm_var}{gcm_time_range}{gcm_lat_range}{gcm_lon_range}"
obs_subset_file = f"{obs_file}?time,lat{obs_lat_range},lon{obs_lon_range},climatology_bounds,crs,{obs_var}{obs_time_range}{obs_lat_range}{obs_lon_range}"
print("GCM Subset: " + gcm_subset_file + "\n")
print("Obs Subset: " + obs_subset_file + "\n")
gcm_subset_dataset = Dataset(gcm_subset_file)
obs_subset_dataset = Dataset(obs_subset_file)
print("GCM Subset Structure: " + str(gcm_subset_dataset.dimensions.items()) + "\n")
print("Obs Subset Structure: " + str(obs_subset_dataset.dimensions.items()))

GCM Subset: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/downscale/MBCn/CMIP6_MBCn/ACCESS-ESM1-5_10/pr_day_MBCn+PCIC-Blend_ACCESS-ESM1-5_historical+ssp126_r1i1p1f1_gn_19500101-21001231.nc?time,lat[181:217],lon[133:169],pr[0:55151][181:217][133:169]

Obs Subset: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/PRISM/dataportal/pr_monClim_PRISM_historical_run1_197101-200012.nc?time,lat[1002:1302],lon[1246:1546],climatology_bounds,crs,pr[0:12][1002:1302][1246:1546]

GCM Subset Structure: dict_items([('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 37), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 37), ('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 55152)])

Obs Subset Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 13), ('bnds', <class 'netCDF4._netCDF4.Dimension'>: name = '

In [46]:
print("GCM subset lats: " + str(gcm_subset_dataset.variables["lat"][:]) + "\n")
print("GCM subset lons: " + str(gcm_subset_dataset.variables["lon"][:]))

GCM subset lats: [56.125      56.20833588 56.29166794 56.375      56.45833588 56.54166794
 56.625      56.70833588 56.79166794 56.875      56.95833588 57.04166794
 57.125      57.20833588 57.29166794 57.375      57.45833588 57.54166794
 57.625      57.70833588 57.79166794 57.875      57.95833588 58.04166794
 58.125      58.20833588 58.29166794 58.375      58.45833588 58.54166794
 58.625      58.70833588 58.79166794 58.875      58.95833588 59.04166794
 59.125     ]

GCM subset lons: [-129.875      -129.79167175 -129.70832825 -129.625      -129.54167175
 -129.45832825 -129.375      -129.29167175 -129.20832825 -129.125
 -129.04167175 -128.95832825 -128.875      -128.79167175 -128.70832825
 -128.625      -128.54167175 -128.45832825 -128.375      -128.29167175
 -128.20832825 -128.125      -128.04167175 -127.95833588 -127.875
 -127.79166412 -127.70833588 -127.625      -127.54166412 -127.45833588
 -127.375      -127.29166412 -127.20833588 -127.125      -127.04166412
 -126.95833588 -126.875   

In [47]:
print("Obs subset lats: " + str(obs_subset_dataset.variables["lat"][:]) + "\n")
print("Obs subset lons: " + str(obs_subset_dataset.variables["lon"][:]))

Obs subset lats: [56.35       56.35833333 56.36666667 56.375      56.38333333 56.39166667
 56.4        56.40833333 56.41666667 56.425      56.43333333 56.44166667
 56.45       56.45833333 56.46666667 56.475      56.48333333 56.49166667
 56.5        56.50833333 56.51666667 56.525      56.53333333 56.54166667
 56.55       56.55833333 56.56666667 56.575      56.58333333 56.59166667
 56.6        56.60833333 56.61666667 56.625      56.63333333 56.64166667
 56.65       56.65833333 56.66666667 56.675      56.68333333 56.69166667
 56.7        56.70833333 56.71666667 56.725      56.73333333 56.74166667
 56.75       56.75833333 56.76666667 56.775      56.78333333 56.79166667
 56.8        56.80833333 56.81666667 56.825      56.83333333 56.84166667
 56.85       56.85833333 56.86666667 56.875      56.88333333 56.89166667
 56.9        56.90833333 56.91666667 56.925      56.93333333 56.94166667
 56.95       56.95833333 56.96666667 56.975      56.98333333 56.99166667
 57.         57.00833333 57.016666

VIII. Put together the parameters for `chickadee.ci`. In the case for `pr`, the `units_bool` parameter is set to `False` in order to avoid converting the PRISM's `mm` units to the PNWNAmet's `mm/day` units.

In [48]:
params = {"gcm_file": gcm_subset_file, "obs_file": obs_subset_file, "gcm_varname": gcm_var, "obs_varname": obs_var, "max_gb": 0.5, 
          "start_date": date(1981, 1, 1), "end_date": date(2010, 12, 31)}
if gcm_var == "pr":
    params["units_bool"] = False
    params["pr_units"] = "mm/day"

## 2. Run `chickadee.ci` and access the output.

I. Run `chickadee.ci`.

In [49]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    params["out_file"] = out_file.name
    output = chickadee.ci(**params)

ERROR:root:Could not read status document.
ERROR:root:Could not parse XML response.


II. Download the output directly from the WPS output URL.

In [50]:
print("Download output from this URL: " + output.get()[0])

Download output from this URL: https://docker-dev03.pcic.uvic.ca/wpsoutputs/f71267b6-7cc5-11ee-80e1-0242ac12000f/output_xzpelu3_.nc
